In [150]:
!pip install pennylane

In [151]:
import numpy as np
import pennylane as qml
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
from collections import Counter

In [152]:
df = pd.read_csv("DatasetExos.csv" ,delimiter="," )
df

,ep (ms),Acc_x,Acc_y,Acc_z,Gyro_x,Gyro_y,Gyro_z,ID,Exercise,Category,Set
0,2019-01-11 15:08:05.200,0.013500,0.977000,-0.071000,-1.8904,2.4392,0.9388,B,bench,heavy,30.0
1,2019-01-11 15:08:05.400,-0.001500,0.970500,-0.079500,-1.6826,-0.8904,2.1708,B,bench,heavy,30.0
2,2019-01-11 15:08:05.600,0.001333,0.971667,-0.064333,2.5608,-0.2560,-1.4146,B,bench,heavy,30.0
3,2019-01-11 15:08:05.800,-0.024000,0.957000,-0.073500,8.0610,-4.5244,-2.0730,B,bench,heavy,30.0
4,2019-01-11 15:08:06.000,-0.028000,0.957667,-0.115000,2.4390,-1.5486,-3.6098,B,bench,heavy,30.0
...,...,...,...,...,...,...,...,...,...,...,...
9004,2019-01-20 17:33:27.000,-0.048000,-1.041500,-0.076500,1.4146,-5.6218,0.2926,E,row,medium,40.0
9005,2019-01-20 17:33:27.200,-0.037000,-1.030333,-0.053333,-2.7684,-0.5854,2.2440,E,row,medium,40.0
9006,2019-01-20 17:33:27.400,-0.060000,-1.031000,-0.082000,2.8416,-5.1342,-0.1220,E,row,medium,40.0
9007,2019-01-20 17:33:27.600,-0.038667,-1.025667,-0.044667,-0.2318,0.2562,1.1220,E,row,medium,40.0


In [153]:
df.columns

Index(['ep (ms)', 'Acc_x', 'Acc_y', 'Acc_z', 'Gyro_x', 'Gyro_y', 'Gyro_z',
       'ID', 'Exercise', 'Category', 'Set'],
      dtype='object')

In [154]:
df = df.drop(columns=["ep (ms)" , "Exercise"  , "ID" ])
df

,Acc_x,Acc_y,Acc_z,Gyro_x,Gyro_y,Gyro_z,Category,Set
0,0.013500,0.977000,-0.071000,-1.8904,2.4392,0.9388,heavy,30.0
1,-0.001500,0.970500,-0.079500,-1.6826,-0.8904,2.1708,heavy,30.0
2,0.001333,0.971667,-0.064333,2.5608,-0.2560,-1.4146,heavy,30.0
3,-0.024000,0.957000,-0.073500,8.0610,-4.5244,-2.0730,heavy,30.0
4,-0.028000,0.957667,-0.115000,2.4390,-1.5486,-3.6098,heavy,30.0
...,...,...,...,...,...,...,...,...
9004,-0.048000,-1.041500,-0.076500,1.4146,-5.6218,0.2926,medium,40.0
9005,-0.037000,-1.030333,-0.053333,-2.7684,-0.5854,2.2440,medium,40.0
9006,-0.060000,-1.031000,-0.082000,2.8416,-5.1342,-0.1220,medium,40.0
9007,-0.038667,-1.025667,-0.044667,-0.2318,0.2562,1.1220,medium,40.0


In [155]:
for i in df.columns:
    if i not in ["Category"]:
        df[i] = pd.to_numeric(df[i])
df

,Acc_x,Acc_y,Acc_z,Gyro_x,Gyro_y,Gyro_z,Category,Set
0,0.013500,0.977000,-0.071000,-1.8904,2.4392,0.9388,heavy,30.0
1,-0.001500,0.970500,-0.079500,-1.6826,-0.8904,2.1708,heavy,30.0
2,0.001333,0.971667,-0.064333,2.5608,-0.2560,-1.4146,heavy,30.0
3,-0.024000,0.957000,-0.073500,8.0610,-4.5244,-2.0730,heavy,30.0
4,-0.028000,0.957667,-0.115000,2.4390,-1.5486,-3.6098,heavy,30.0
...,...,...,...,...,...,...,...,...
9004,-0.048000,-1.041500,-0.076500,1.4146,-5.6218,0.2926,medium,40.0
9005,-0.037000,-1.030333,-0.053333,-2.7684,-0.5854,2.2440,medium,40.0
9006,-0.060000,-1.031000,-0.082000,2.8416,-5.1342,-0.1220,medium,40.0
9007,-0.038667,-1.025667,-0.044667,-0.2318,0.2562,1.1220,medium,40.0


In [156]:
df = df.reset_index()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9009 entries, 0 to 9008
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   index     9009 non-null   int64  
 1   Acc_x     9005 non-null   float64
 2   Acc_y     9009 non-null   float64
 3   Acc_z     9008 non-null   float64
 4   Gyro_x    9008 non-null   float64
 5   Gyro_y    9002 non-null   float64
 6   Gyro_z    9009 non-null   float64
 7   Category  9003 non-null   object 
 8   Set       8999 non-null   float64
dtypes: float64(7), int64(1), object(1)
memory usage: 633.6+ KB


In [157]:
df = df.dropna()
df = df.drop(columns=["index"])

In [158]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8985 entries, 0 to 9008
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Acc_x     8985 non-null   float64
 1   Acc_y     8985 non-null   float64
 2   Acc_z     8985 non-null   float64
 3   Gyro_x    8985 non-null   float64
 4   Gyro_y    8985 non-null   float64
 5   Gyro_z    8985 non-null   float64
 6   Category  8985 non-null   object 
 7   Set       8985 non-null   float64
dtypes: float64(7), object(1)
memory usage: 631.8+ KB


In [159]:
df

,Acc_x,Acc_y,Acc_z,Gyro_x,Gyro_y,Gyro_z,Category,Set
0,0.013500,0.977000,-0.071000,-1.8904,2.4392,0.9388,heavy,30.0
1,-0.001500,0.970500,-0.079500,-1.6826,-0.8904,2.1708,heavy,30.0
2,0.001333,0.971667,-0.064333,2.5608,-0.2560,-1.4146,heavy,30.0
3,-0.024000,0.957000,-0.073500,8.0610,-4.5244,-2.0730,heavy,30.0
4,-0.028000,0.957667,-0.115000,2.4390,-1.5486,-3.6098,heavy,30.0
...,...,...,...,...,...,...,...,...
9004,-0.048000,-1.041500,-0.076500,1.4146,-5.6218,0.2926,medium,40.0
9005,-0.037000,-1.030333,-0.053333,-2.7684,-0.5854,2.2440,medium,40.0
9006,-0.060000,-1.031000,-0.082000,2.8416,-5.1342,-0.1220,medium,40.0
9007,-0.038667,-1.025667,-0.044667,-0.2318,0.2562,1.1220,medium,40.0


In [160]:
from sklearn.preprocessing import StandardScaler

Y = df["Category"]
X = df.drop(columns="Category")
scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=42)
X_train

array([[-9.33094536e-03,  4.37151252e-01,  1.03481357e+00, ...,
         4.06094710e-01,  1.58536131e-02, -3.32021248e-02],
       [-9.55639800e-03,  8.37745599e-01, -7.61842326e-01, ...,
         1.03271725e+00, -7.65422900e-02,  1.43817194e+00],
       [-9.43612515e-03, -1.16193808e+00, -5.00455934e-01, ...,
         8.30536988e-02, -1.56710909e-03,  1.65887805e+00],
       ...,
       [-9.27010145e-03,  6.51239977e-01,  1.55758635e+00, ...,
         1.15161906e-02, -8.48360905e-03,  9.96759718e-01],
       [-9.61189645e-03,  6.27127595e-01, -5.92592133e-01, ...,
         5.39485559e-01,  6.50182663e-02,  1.40138759e+00],
       [-9.49225248e-03, -1.21071085e+00, -4.92944830e-01, ...,
        -1.26739529e-01, -2.54931281e-03, -1.57814489e+00]])

In [161]:
y_train

,Category
4108,heavy
6874,medium
3750,heavy
469,heavy
5253,medium
...,...
5749,medium
5205,medium
5404,heavy
864,medium


In [162]:
_qubits = 4
dev = qml.device('default.qubit', wires=7)
def encode_data(data):
    encoded_states = []
    for point in data:
        encoded_state = np.zeros((_qubits,), dtype=int)
        for i in range(_qubits):
            if point[i] > 0.5:
                encoded_state[i] = 1
        encoded_states.append(encoded_state)
    return np.array(encoded_states)

In [163]:
X_train_encoded = encode_data(X_train)
X_test_encoded = encode_data(X_test)

In [164]:
len(X_train_encoded)

8086

In [165]:
X_test_encoded

array([[0, 0, 0, 1],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       ...,
       [0, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 0, 0]])

In [166]:
y_train

,Category
4108,heavy
6874,medium
3750,heavy
469,heavy
5253,medium
...,...
5749,medium
5205,medium
5404,heavy
864,medium


In [167]:
y_train.value_counts()

,count
Category,
medium,3969
heavy,3119
standing,534
sitting,463
heav,1


In [168]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)
y_train_encoded = encoder.fit_transform(np.array(y_train).reshape(-1, 1))

In [ ]:
@qml.qnode(dev)
def quantum_feature_map(x):
    for i in range(_qubits):
        qml.RY(x[i] * np.pi, wires=i)
    return [qml.expval(qml.PauliZ(i)) for i in range(_qubits)]

def train_qsvm(x_train, y_train):
    weights = np.random.randn(_qubits, y_train.shape[1])
    learning_rate = 0.1

    for epoch in range(40):
        print(f"epoch {epoch} in progress")
        for i in range(len(x_train)):
            features = quantum_feature_map(x_train[i])
            for class_idx in range(y_train.shape[1]):
                prediction = np.dot(weights[:, class_idx], features)
                error = y_train[i, class_idx] - (1 if prediction > 0 else 0)
                weights[:, class_idx] += learning_rate * error  

    return weights
weights = train_qsvm(X_train, y_train_encoded)


epoch 0 in progress
epoch 1 in progress
epoch 2 in progress
epoch 3 in progress
epoch 4 in progress
epoch 5 in progress
epoch 6 in progress
epoch 7 in progress
epoch 8 in progress
epoch 9 in progress
epoch 10 in progress
epoch 11 in progress
epoch 12 in progress
epoch 13 in progress
epoch 14 in progress
epoch 15 in progress
epoch 16 in progress
epoch 17 in progress
epoch 18 in progress
epoch 19 in progress
epoch 20 in progress
epoch 21 in progress
epoch 22 in progress
epoch 23 in progress
epoch 24 in progress
epoch 25 in progress
epoch 26 in progress
epoch 27 in progress
epoch 28 in progress
epoch 29 in progress
epoch 30 in progress
epoch 31 in progress
epoch 32 in progress
epoch 33 in progress
epoch 34 in progress
epoch 35 in progress
epoch 36 in progress
epoch 37 in progress
epoch 38 in progress
epoch 39 in progress


In [170]:
def predict(x, weights):
    features = quantum_feature_map(x)
    predictions = np.dot(weights.T, features)
    class_index = np.argmax(predictions)
    return encoder.categories_[0][class_index]

y_pred = [predict(x , weights) for x in X_test]
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.33


In [171]:
y_test

,Category
1934,medium
96,heavy
768,heavy
3092,medium
4212,heavy
...,...
2442,heavy
1407,medium
5738,medium
2620,heavy
